### Training

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
import torch.nn as nn
import torch
from tqdm.auto import tqdm
import random
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from commom import load_jsonl, save_jsonl

from transformers import (
    AdamW,
    get_scheduler,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
)

In [3]:
class NLIDataset(Dataset):
    def __init__(self, data_list, max_length=512, model_name="bert-base-multilingual-cased"):
        self.d_list = data_list
        self.len = len(self.d_list)
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.label2index = {
            'SPAM': 0,
            'EDM': 1,
            'HAM': 2
        }

    def __getitem__(self, index):
        data = self.d_list[index]
        context = data['context']
        label = data['label']
        
        processed_sample = dict()
        processed_sample['labels'] = torch.tensor(self.label2index[label])
        tokenized_input = self.tokenizer(context,
                                         max_length=self.max_length,
                                         padding='max_length', 
                                         truncation=True,
                                         return_tensors="pt")
        
        input_items = {key: val.squeeze() for key, val in tokenized_input.items()}
        processed_sample.update(input_items)
        return processed_sample

    def __len__(self):
        return self.len

In [11]:
train_list = load_jsonl('datasets/cross_datasets_section_5/train.jsonl')
dev_list = load_jsonl('datasets/cross_datasets_section_5/dev.jsonl')

Load Jsonl: datasets/cross_datasets_section_5/train.jsonl


259517it [00:15, 16491.17it/s]


Load Jsonl: datasets/cross_datasets_section_5/dev.jsonl


32439it [00:01, 16656.83it/s]


In [12]:
train_dataset = NLIDataset(train_list)
dev_dataset = NLIDataset(dev_list)

In [13]:
train_list[0]

{'index': '64879',
 'md5sum': '30c435d0dc314f4450b94e37da54fc3c',
 'label': 'EDM',
 'context': 'oracle oracleview message web browser mysql webinar 400x query acceleration oracle mysql heatwave march       pst register today 400x query acceleration mysql heatwave heatwave new memory query accelerator mysql database service available oracle cloud heatwave accelerate performance large multi tb datasets scale across  000s core benchmark result show heatwave accelerates mysql query 400x   cost amazon redshift join u learn leverage heatwave accelerate mysql query register today register today march     pt stay connect term use privacysubscriptionsunsubscribecontact u copyright emojipic  oracle affiliate right reserve oracle corporation worldwide headquarters  oracle way austin tx  united state'}

In [14]:
train_dataset[0]

{'labels': tensor(1),
 'input_ids': tensor([  101, 14480, 19478, 14480, 19478, 30512, 30514, 12998, 91597, 15127,
         10107, 11703, 10161, 12998, 39735, 11176, 10686, 59599, 10157, 13621,
         32942, 58656, 14480, 19478, 15127, 10107, 11703, 10161, 33955, 85164,
         78466, 97764, 10123, 47424, 18745, 11176, 10686, 59599, 10157, 13621,
         32942, 58656, 15127, 10107, 11703, 10161, 33955, 85164, 33955, 85164,
         10751, 25296, 59599, 10157, 13621, 32942, 52264, 10667, 15127, 10107,
         11703, 10161, 11254, 11989, 14579, 14480, 19478, 78394, 33955, 85164,
         13621, 32942, 64474, 14432, 12077, 21247,   188, 10457, 11165, 14488,
         10107, 19707, 15130, 10259, 10107, 27362, 86696, 22719, 14493, 11897,
         33955, 85164, 13621, 32942, 64474, 10107, 15127, 10107, 11703, 10161,
         59599, 10157, 11176, 10686, 18849, 28149, 22742, 10680, 16119, 12961,
         23707,   189, 42671, 25864, 12276, 33955, 85164, 13621, 32942, 64474,
         15127, 1

In [15]:
# note: 3 classes
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.device_count() >1:
    model = nn.DataParallel(model,device_ids=[0])
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [16]:
train_batch_size=40
learning_rate=2e-5 
train_epochs=5

optimizer = AdamW(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=train_batch_size)
print(len(train_dataloader))
print(len(dev_dataloader))

6488
811


In [17]:
for batch_index, batch_dict in enumerate(train_dataloader):
    print(batch_dict)
    break

{'labels': tensor([0, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
        1, 2, 1, 0, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1]), 'input_ids': tensor([[  101, 52770, 41784,  ...,     0,     0,     0],
        [  101,  7860,  3457,  ...,  4310,  6180,   102],
        [  101,  8376,  2551,  ...,     0,     0,     0],
        ...,
        [  101, 14535, 79515,  ...,     0,     0,     0],
        [  101,  8148,  7290,  ...,     0,     0,     0],
        [  101,  7150,  3910,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [18]:
## 進度條
num_training_steps = train_epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

## 設定warmup
lr_scheduler = get_scheduler(
  "linear",
  optimizer=optimizer,
  num_warmup_steps=10,
  num_training_steps=num_training_steps
)

## start training
for epoch in range(train_epochs):
    model.train()
    for batch_index, batch_dict in enumerate(train_dataloader):
        
        input_items = {key: val.to(device) for key, val in batch_dict.items()}
#         del input_items['token_type_ids'] ## bart不需要這個
        
        optimizer.zero_grad()
        outputs = model(**input_items)
        
        loss = outputs.loss
        if torch.cuda.device_count() >1: ##多GPU的情況要對loss求平均
            loss = loss.mean()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)
        
        if batch_index % 500 ==0:
            print('epoch: ', epoch, '  loss: ', loss)
            
    model.eval()
    predictions = []
    references = []
    with torch.no_grad():
        for batch_index, batch_dict in enumerate(dev_dataloader):
            input_items = {key: val.to(device) for key, val in batch_dict.items()}
            outputs = model(**input_items)

            predictions += outputs.logits.argmax(dim=-1).tolist()
            references += batch_dict['labels'].tolist()

    accuracy = accuracy_score(references, predictions)
    f1 = f1_score(references, predictions,average='macro')
    print('acc: ', accuracy)
    print('f1: ', f1)
    
    ## save model
    save_path = 'datasets/cross_datasets_section_5/epoch_' + str(epoch+1)
    if torch.cuda.device_count() >1:
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(save_path)
    else:
        model.save_pretrained(save_path)

  0%|          | 0/32440 [00:00<?, ?it/s]

epoch:  0   loss:  tensor(1.1214, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1839, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1751, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0189, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0043, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0876, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0029, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0027, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1168, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0033, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0021, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1271, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0468, device='cuda:0', g

# Test

In [23]:
test_list = load_jsonl('datasets/cross_datasets_section_4/test.jsonl')
test_dataset = NLIDataset(test_list)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=512)

Load Jsonl: datasets/cross_datasets_section_4/test.jsonl


32440it [00:01, 16886.33it/s]


In [24]:
# note: 3 classes
model_path = 'datasets/cross_datasets_section_4/epoch_5'
# model_path = 'models/Mail_Classifier/epoch_5'
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [25]:
model.eval()
predictions = []
references = []
num_steps = len(test_dataloader)
progress_bar = tqdm(range(num_steps))

with torch.no_grad():
    for batch_index, batch_dict in enumerate(test_dataloader):
        input_items = {key: val.to(device) for key, val in batch_dict.items()}
        outputs = model(**input_items)

        predictions += outputs.logits.argmax(dim=-1).tolist()
        references += batch_dict['labels'].tolist()
        progress_bar.update(1)

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

  0%|          | 0/64 [00:00<?, ?it/s]

acc:  0.9932799013563501
f1:  0.9881536796146634


In [26]:
# 4
print(model_path)
print(classification_report(references, predictions))

datasets/cross_datasets_section_4/epoch_5
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3681
           1       1.00      1.00      1.00     21835
           2       0.99      0.99      0.99      6924

    accuracy                           0.99     32440
   macro avg       0.99      0.99      0.99     32440
weighted avg       0.99      0.99      0.99     32440



## inference

In [1]:
import torch.nn as nn
import torch
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from commom import load_jsonl, save_jsonl

from transformers import (
    BartTokenizer,
    BartForSequenceClassification,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
)

In [2]:
test_list = load_jsonl('datasets/test.jsonl')

Load Jsonl: datasets/test.jsonl


32440it [00:01, 17159.63it/s]


In [10]:
model = AutoModelForSequenceClassification.from_pretrained('models/Mail_Classifier_10/epoch_5', num_labels=3) 
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")  

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.device_count() >1:
    model = nn.DataParallel(model,device_ids=[0,1])
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
test_num = 10

# subject = test_list[test_num]['subject']
context = test_list[test_num]['context']
label = test_list[test_num]['label']

In [12]:
tokenized_input = tokenizer(context,
                            max_length=512,
                            truncation=True,
                            return_tensors="pt")
class_number = {'SPAM':0, 'EDM':1, 'HAM':2}
model.eval()
with torch.no_grad():
    input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    del input_items['token_type_ids'] ## bart不需要這個
    
    outputs = model(**input_items)
    prediction = outputs.logits.argmax(dim=-1)
    print(type(int(prediction)))
    
    
    # print('主旨: ', subject)
    print('內文: ', context)
    print('label: ', class_number[label], label)
    print('predict: ', int(prediction), list(class_number.keys())[list(class_number.values()).index(int(prediction))])
    
    # if int(prediction) == 0:
    #     print('predict: ham')
    # else:
    #     print('predict: spam')

<class 'int'>
內文:  圖片 顯 現時 按此 下載 文宣 網頁 不想 收到 本行 信用卡 行銷 訊息 按此 登入 網路 銀行 信用卡 專區 進行 行銷 訊息 取消 訂閱 設定 電子 訊息 內容 包括 附件 合作金庫 銀行 股份 有限公司 傳送 電子 訊息 內容 機密性 經由 公司 授權 方可 利用 電子 訊息 指定 收件 任何人 公司 電子 訊息 內容 審閱 傳送 散佈 揭露 重製 指定 收件 通知 並請 刪除 電子 訊息 內容 謝謝您 合作 電子 訊息 內容 變更 網際網路 保證 電子 訊息 內容 完整性 公司 變更 修改 竄改 偽造 電子 訊息 內容 恕 不負 責任 網路 通訊 含有 電腦病毒 收件 應 自行 確認 郵件 內容 損害 公司 恕 負責
label:  1 EDM
predict:  1 EDM


## eval

In [13]:
from sklearn.metrics import classification_report

class_number = {'SPAM':0, 'EDM':1, 'HAM':2}

predictions = []
references = []

for batch_index, batch_dict in enumerate(test_list[:200]):
    input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    outputs = model(**input_items)

    predictions += outputs.logits.argmax(dim=-1).tolist()

    references += [class_number[batch_dict['label']]]
    # print(reference[:10])
print(len(predictions))

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

print("----------")
classification_report(references, predictions, target_names=class_number)

200
acc:  0.725
f1:  0.2801932367149758
----------


/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n        SPAM       0.00      0.00      0.00        22\n         EDM       0.72      1.00      0.84       145\n         HAM       0.00      0.00      0.00        33\n\n    accuracy                           0.73       200\n   macro avg       0.24      0.33      0.28       200\nweighted avg       0.53      0.72      0.61       200\n'

In [ ]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
import torch.nn as nn
import torch
from tqdm.auto import tqdm
import random
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from common import load_jsonl, save_jsonl

from transformers import (
    AdamW,
    get_scheduler,
    BartTokenizer,
    BartForSequenceClassification,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
#     AutoModelForMaskedLM,
)

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification, BertTokenizerFast

In [ ]:
eval_list = load_jsonl('test-data.jsonl')
eval_dataset = NLIDataset(eval_list)
eval_dataset[0]
test_dataloader = DataLoader(eval_dataset, shuffle=True, batch_size=train_batch_size)

In [ ]:
predictions = []
references = []

for batch_index, batch_dict in enumerate(test_dataloader):
    input_items = {key: val.to(device) for key, val in batch_dict.items()}
    outputs = model(**input_items)

    predictions += outputs.logits.argmax(dim=-1).tolist()
    references += batch_dict['labels'].tolist()

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

In [ ]:
num = 3
print("the sentence is ")
print()
print(input_items[3])
outputs = model(**input_items[3])
predictions = outputs.logits.argmax(dim=-1).tolist()
print("predict : ", predictions)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_list = ["0", "1"]
tokenizer = BertTokenizerFast.from_pretrained("Transformers Trainer")
config = BertConfig.from_pretrained("Transformers Trainer", finetuning_task="cola")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

In [ ]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.logits.argmax(dim=-1).item()]}")